In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime

from common import *
from mcs_shared import (
    SNOBAL_DIR, PRECIP_DIR, MC_ALS, mcs_snotel_csv, 
    SnotelPointData, get_station_pixel_factors
)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
use_hvplot()

In [ ]:
RESOLUTION = 10 # meters

# Description
Compare the incoming precipitation amounts from the measurements at the SNOTEL station to pixel location in HRRR.

## SNOTEL 

In [ ]:
mcs_snotel_point = SnotelPointData("637:ID:SNTL", "MCS")

In [ ]:
mcs_snotel_mt = mcs_snotel_csv()
mcs_snotel_mt['MCS depth diff'] = mcs_snotel_mt['MCS depth'].diff()

In [ ]:
mcs_snotel_mt.head()

## iSnobal

In [ ]:
mcs_snobal_snow = pd.read_csv(f'{MC_ALS}/MCS-iSnobal-snow_depth.csv', parse_dates=['time'], index_col=['time'])
mcs_snobal_snow['iSnobal depth diff'] = mcs_snobal_snow['iSnobal depth'].diff()
# mcs_snobal_snow.head()

## HRRR data

In [ ]:
precip = {}
precip['2022'] = pd.read_csv(f'{PRECIP_DIR}/2022.csv', parse_dates=['time'], index_col=['time'])
precip['2023'] = pd.read_csv(f'{PRECIP_DIR}/2023.csv', parse_dates=['time'], index_col=['time'])
precip['2024'] = pd.read_csv(f'{PRECIP_DIR}/2024.csv', parse_dates=['time'], index_col=['time'])
precip['2025'] = pd.read_csv(f'{PRECIP_DIR}/2025.csv', parse_dates=['time'], index_col=['time'])

for year, data in precip.items():
    data['HRRR precip diff'] = data['HRRR precip'].diff()

    data = pd.merge(
        data, 
        mcs_snobal_snow.loc[data.index.min():data.index.max()],
        left_index=True, right_index=True, how='inner'
    ).merge(
        mcs_snotel_mt.loc[data.index.min():data.index.max()],
        left_index=True, right_index=True, how='inner'
    )

    data['precip factor'] = (data['HRRR precip'] / data['MCS precip accumulated'])
    data['precip diff factor'] = (data['HRRR precip diff'] / data['MCS precip diff'])
    data['depth factor'] = (data['MCS depth'] / data['iSnobal depth'])

    precip[year] = data

In [ ]:
precip['2022'].head()

## Factors
### Statistics for 1 and 99th percentile

In [ ]:
precip_desc = []
depth_desc = []

for year, data in precip.items():   
    precip_desc.append(data['precip factor'].replace([np.inf, -np.inf], np.nan).describe(percentiles=[.01, 0.5, 0.99]))
    depth_desc.append(data['depth factor'].replace([np.inf, -np.inf], np.nan).describe(percentiles=[.01, 0.5, 0.99]))

### Precip

In [ ]:
pd.concat(precip_desc, axis=1, keys=precip.keys())

### Depth

In [ ]:
pd.concat(depth_desc, axis=1, keys=precip.keys())

## Filter to only days with accumulating precip and snow on the ground above 0.5

In [ ]:
all = pd.concat(precip.values())
for year, data in precip.items():
    precip[year] = data[(data['HRRR precip diff'] > 1) | (data['MCS precip diff'] > 1)]
    # precip[year] = data[(data['iSnobal depth'] > .75) | (data['MCS depth'] > .75)]

### Statistics for 1 and 99th percentile

In [ ]:
depth_desc = []
precip_desc = []

for year, data in precip.items():   
    precip_desc.append(data['precip factor'].replace([np.inf, -np.inf], np.nan).describe(percentiles=[.01, 0.5, 0.99]))
    depth_desc.append(data['depth factor'].replace([np.inf, -np.inf], np.nan).describe(percentiles=[.01, 0.5, 0.99]))

### Precip

In [ ]:
pd.concat(precip_desc, axis=1, keys=precip.keys())

### Depth

In [ ]:
pd.concat(depth_desc, axis=1, keys=precip.keys())

In [ ]:
def plot_year(data, year):
    scatter_opts=dict(size=15)
    
    return hv.Scatter(
        data, kdims=['HRRR precip diff'], 
        vdims=['MCS precip diff', 'precip diff factor', 'depth factor'],
    ).opts(
        title=year,
        ylabel='MCS Precip difference (mm)',
        xlabel='HRRR Precip difference (mm)',
        width=600, height=600, aspect='equal',
        tools=['hover'],
        xlim=(-1, 80), ylim=(-1, 80),
        # size=hv.dim('precip factor') * 10,
        cmap='coolwarm_r', clim=(0.1, 2.1), color='precip diff factor',
        radius=dim('depth factor').bin(np.arange(0.8, 2, 0.1))
    )

In [ ]:
plot = [
    plot_year(precip[year], year) for year in precip.keys()
]

hv.Layout(plot).cols(2)

# HV plot

## SNOTEL 

In [ ]:
STATION_LABEL = 'Station'
COLORS = {
        'iSnobal': 'cornflowerblue',
        STATION_LABEL: 'seagreen',    
}

In [ ]:
# HV_PLOT_OPTS['ylabel'] = 'Snow Precip Accumulated (mm)'
SNOTEL_LINE=dict(line_width=2, line_dash=(4, 2))

In [ ]:
def plot_precip(data, year):
    return(
        hv.Scatter(
            data, kdims='index', vdims=['HRRR precip'], label='HRRR precip'
        ).opts(color=COLORS['iSnobal'], size=5, ylim=(0, 1200), tools=['hover']) * \
        hv.Scatter(
            data, kdims='index', vdims=['MCS precip accumulated'], label='MCS precip'
        ).opts(color=COLORS['Station'], size=5, tools=['hover']) * \
        hv.Scatter(
            data, kdims='index', vdims=['MCS SWE'], label='MCS SWE'
        ).opts(color='orange', size=5, tools=['hover'])
    ).opts(
        title=year,
        tools=['hover'],
        width=600, height=600,
        ylabel='Snow Precip Accumulated (mm)',
        xlabel='Time',
    )

hv.Layout(
    [plot_precip(data, year) for year, data in precip.items()]
).cols(2).opts(title='Precip', shared_axes=False)

In [ ]:
def plot_depth(data, year):
    return (hv.Curve(
            data, kdims=['index'], vdims=['iSnobal depth'], label='iSnobal'
        ).opts(ylim=(-0.1, 3), color='coral', **LINE_STYLE) * \
        hv.Curve(
            data, kdims=['index'], vdims=['MCS depth'], label='Station'
        ).opts(ylim=(-0.1, 3), color='purple', **SNOTEL_LINE)
    ).opts(
        title=year,
        tools=['hover'],
        width=600, height=600,
        ylabel='Depth (m)',
        xlabel='Time',
    )

In [ ]:
hv.Layout([
    plot_depth(data, year) for year, data in precip.items()
]).opts(shared_axes=False).cols(2)

In [ ]:
precip_factors = get_station_pixel_factors(RESOLUTION, mcs_snotel_point)

In [ ]:
def plot_factors(data, year, als_factors):
    min_date = data.index.min()
    max_date = data.index.max()
    als_values = [
        value for value in als_factors if value[0] > min_date and value[0] < max_date
    ]
    
    return (
            hv.HLines([1, .75, .5]).opts(color='grey', line_width=2, line_dash='dashed') * \
            data.hvplot.scatter(
                x='index', y='precip factor', label='precip', size=5
            ).opts(color='purple', ylim=(-0.1, 2), size=5) * \
            data.hvplot.scatter(
                x='index', y='depth factor', label='depth',  size=5
            ).opts(color='orange', size=5) * \
            hv.Scatter(als_values, label='ALS factors').opts(color='royalblue',size=8, tools=['hover'])
        ).opts(
            title=year,
            tools=['hover'],
            width=800, height=800,
            ylabel='factors',
            xlabel='Time',
        )

In [ ]:
hv.Layout([
    plot_factors(data, year, precip_factors) for year, data in precip.items()
]).opts(shared_axes=False, title='Factors').cols(2)

In [ ]:
als_days = all.loc[[d[0] for d in precip_factors]]['depth factor']

In [ ]:
als_mcs = pd.DataFrame({
    'ALS': [d[1] for d in precip_factors],
    'MCS': all.loc[[d[0] for d in precip_factors]]['depth factor'].values,
    'MCS precip': all.loc[[d[0] for d in precip_factors]]['precip factor'].values,
    'date': [d[0] for d in precip_factors]
}).set_index('date')

In [ ]:
als_mcs

In [ ]:
hv.Scatter(als_mcs, kdims=['MCS'], vdims=['ALS']).opts(size=10, width=400, height=400, title='MCS vs ALS depth factor')

In [ ]:
def plot_factors(data, year, als_factors):
    return hv.Scatter(
            data, kdims='MCS depth', vdims=['precip factor', 'index', 'depth factor', 'iSnobal depth'], label=year
        ).opts(
        size=10, 
        width=800, height=800, 
        ylabel='Precip Factor', 
        xlabel='MCS Depth', 
        tools=['hover'],
        hover_tooltips=[
            ("Date", "@index"), 
            ("iSnobal", "@{iSnobal depth}"), 
            ("MCS", "@{MCS depth}"), 
            ("Depth", "@{depth factor}"), 
            ("Precip", "@{precip factor}")
        ]
    )
hv.Overlay([plot_factors(data, year, precip_factors) for year, data in precip.items()])

In [ ]:
def plot_factors(data, year, als_factors):
    return hv.HLines([1]).opts(color='grey', line_width=2, line_dash='dashed') * \
        hv.Scatter(
            data, kdims='MCS depth', vdims=['depth factor', 'index', 'precip factor', 'iSnobal depth'], label=year
        ).opts(
        size=10, 
        width=800, height=800, 
        ylabel='Depth Factor', 
        xlabel='MCS Depth', 
        tools=['hover'],
        hover_tooltips=[
            ("Date", "@index"), 
            ("iSnobal", "@{iSnobal depth}"), 
            ("MCS", "@{MCS depth}"), 
            ("Depth", "@{depth factor}"), 
            ("Precip", "@{precip factor}")
        ]
    )
    
hv.Overlay([plot_factors(data, year, precip_factors) for year, data in precip.items()])

In [ ]:
def plot_factors(data, year, als_factors):
    return hv.Scatter(
            data, kdims='depth factor', vdims=['precip factor', 'index', 'MCS depth', 'iSnobal depth'], label=year
        ).opts(
        size=10, 
        width=800, height=800, 
        ylabel='Precip Factor', 
        xlabel='Depth Factor', 
        tools=['hover'],
        hover_tooltips=[
            ("Date", "@index"), 
            ("iSnobal", "@{iSnobal depth}"), 
            ("MCS", "@{MCS depth}"), 
            ("Depth", "@{depth factor}"), 
            ("Precip", "@{precip factor}")
        ]
    )
    
hv.Overlay([plot_factors(data, year, precip_factors) for year, data in precip.items()])